In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
# import tabula
import composition_stats as cs
from pandasql import sqldf
import math
from scipy.stats import chi2
from scipy import stats
from matplotlib import pyplot as plt
import re

In [ ]:
sample_md = pd.read_excel(r"C:\Users\edwar\Desktop\Melbourne\research_project\HGMA_data\sample_metadata.xlsx")
abund_data = pd.read_csv(r"C:\Users\edwar\Desktop\Melbourne\research_project\HGMA_data\vect_atlas.csv\vect_atlas.csv")
corr_taxa = pd.read_csv(r"C:\Users\edwar\Desktop\Melbourne\research_project\HGMA_data\corresponding_taxa.csv")

In [ ]:
corr_taxa = corr_taxa[~corr_taxa['name'].str.contains('unclassified')].reset_index(drop = True)
multi_taxa = corr_taxa[corr_taxa['name'].str.contains(r"\s\d$")].reset_index(drop = True)
multi_taxa['name'] = multi_taxa['name'].apply(lambda x : x[:-2])
multi_taxa_lst = list(multi_taxa['name'].unique())

## remember to update the disease type and BioProject code
dataset_study = "PRJEB1786"   # PRJEB1786: should change the dataset here

study2_metadata = sample_md[sample_md['BioProject'] == dataset_study].reset_index(drop = True)    
metadata = study2_metadata
metadata = metadata[metadata['Disease'].isin(['T2D', 'Healthy', 'NGT'])].reset_index(drop = True)
sp_id = list(metadata['sample.ID'])
rawdata = abund_data[sp_id + ['Unnamed: 0']]
rawdata = rawdata.merge(corr_taxa, how = 'left', left_on = ['Unnamed: 0'], right_on = ['id']).drop(['Unnamed: 0', 'id'], axis = 1)
rawdata = rawdata[rawdata['name'].isna() == False].reset_index(drop = True)

taxa_lst = list(rawdata['name'])
rawdata_v1 = rawdata.drop(['name'], axis = 1).T
rawdata_v1.columns = taxa_lst
rawdata_v1 = rawdata_v1.reset_index().rename(columns = {'index':'sample_id'})
rawdata_v1 = rawdata_v1.merge(metadata[['sample.ID', 'Disease']], how = 'left', left_on = ['sample_id'], right_on = ['sample.ID']).drop(['sample.ID'], axis = 1)

rawdata_v1 = rawdata_v1.rename(columns = {'Blautia coccoides == Blautia producta':'Blautia producta'})

In [ ]:
# filtering out the inconsistent naming of taxa
def check_main_species(lst):
    contain_main_species = False
    integers = [str(i) for i in range(1, 10)]
    for sub in lst:
        if sub[-1] not in integers:
            contain_main_species = True
            break
    # True means it contains main species, False means all are subspecies which should sum them up in the next step
    return contain_main_species

for taxa in multi_taxa_lst:
    if 'subtype' not in taxa.lower():
        cols = list(rawdata_v1.columns)
        rawdata_v1[taxa] = 0
        alltaxa_columns = [col for col in cols if taxa in col]
        if len(alltaxa_columns) == 1:
            rawdata_v1[taxa] = rawdata_v1[alltaxa_columns[0]]

        elif check_main_species(alltaxa_columns):
            alltaxa_columns.remove(taxa)
            rawdata_v1 = rawdata_v1.drop(alltaxa_columns, axis = 1)

        else:
            rawdata_v1[taxa] = rawdata_v1[alltaxa_columns].sum(axis = 1)
            rawdata_v1 = rawdata_v1.drop(alltaxa_columns, axis = 1)

In [ ]:
drop_cols = [taxa for taxa in list(rawdata_v1.columns)[1:-1] if (len(rawdata_v1[taxa].unique()) == 1) & (taxa != 'Disease')]
rawdata_v2 = split_processing_single(rawdata_v1, thres = 0.0001); print(rawdata_v2.shape)